# Problem: Write AlexNet from Scratch

### Problem Statement
Implement the **AlexNet architecture** in PyTorch by completing the required sections. The model should include convolutional layers, ReLU activations, pooling layers, and fully connected layers to process image data for classification tasks.

### Requirements

1. **Define the AlexNet Architecture**:
   - **Feature Extractor (Convolutional Base)**:
     - Stack convolutional layers with appropriate kernel sizes, strides, and paddings.
     - Use `nn.ReLU` as the activation function after each convolution.
     - Apply `nn.MaxPool2d` after selected layers to reduce spatial dimensions.
   - **Classifier (Fully Connected Layers)**:
     - Flatten the output from the convolutional base.
     - Add fully connected layers with ReLU activations and dropout for regularization.
     - End with a final linear layer projecting to the number of output classes.

2. **Implement the Forward Method**:
   - Pass the input image through the convolutional base.
   - Flatten the feature map output to a vector.
   - Pass it through the fully connected classifier to produce final predictions.

3. **Weight Initialization**:
   - Initialize weights of convolutional and linear layers using a normal distribution.
   - Set biases to zero.

### Constraints
- Assume input images are RGB with shape **(3, 224, 224)**.
- Ensure the model is compatible with **batch processing**.
- The final layer output size should match the number of target classes (e.g., 10 for CIFAR-10).
- Avoid using any pretrained models or high-level wrappers like `torchvision.models`.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [ ]:
# Load data
transform = transforms.Compose([
    transforms.Resize(224),  # Resize to AlexNet input
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
# Define AlexNet
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):  # Adjusted for CIFAR-10
        ...

    def forward(self, x):
        ...



In [ ]:
# --- Training setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)



In [ ]:
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")



In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')
